In [ ]:
using Pkg
Pkg.activate("..")
using PIML4Power
using Flux
using Plots
using DataFrames

In [ ]:
# import data
data, mat, id = load_data("../data/test_data_set.h5");
id_batch = collect(1:10:200)
opt = ADAM(0.01)

In [ ]:
# create all-to-all structure
epsilon = generate_full_line_list(id.Nbus)

# or create n-nearest neighbour
#epsilon = generate_neighbour_list(data.epsilon, 5)

In [ ]:
# initialize the grid parameters
Nline = size(epsilon, 1)
gamma = 2 * ones(Nline)
beta = 4 * ones(Nline)
gsh = 1E-1 * ones(id.Nbus)
bsh = 1E-1 * ones(id.Nbus);

In [ ]:
Nepoch = 20
Ninter = 500
epsilon, beta, gamma, bsh, gsh = train_n_update_V2S_map!(data, epsilon, id,
    id_batch, opt, PIML4Power.exp_param, PIML4Power.red_param, beta, gamma,
    bsh, gsh, Ninter = Ninter, Nepoch = Nepoch, Nsubepoch = 4000, b_thres = 0.1);

In [ ]:
# first compute them from parametrization
b, g, bsh, gsh = PIML4Power.exp_param((beta, gamma, bsh, gsh));

In [ ]:
# compare the lines detected by the method with the real ones
line_id1 = data.epsilon[:,1]*1000 + data.epsilon[:,2] # here we loosely convert id1 x id2 -> id by multiply id1 by a large const
line_id2 = epsilon[:,1]*1000 + epsilon[:,2]
unique_line_id = sort(unique([line_id1; line_id2]))
unique_b = [NaN for i =1:length(unique_line_id), j=1:2]
unique_g = [NaN for i =1:length(unique_line_id), j=1:2]

# populate
for i = 1:length(unique_line_id)
    id1 = findall(line_id1 .== unique_line_id[i])
    id2 = findall(line_id2 .== unique_line_id[i])
    if !isempty(id1)
        unique_b[i, 1] = data.b[id1][1]
        unique_g[i, 1] = data.g[id1][1]
    end
    if !isempty(id2)
        unique_b[i, 2] = b[id2][1]
        unique_g[i, 2] = g[id2][1]
    end
end

# print as a table
id2 = mod.(unique_line_id,1000)
id1 = Int64.((unique_line_id - id2) / 1000)
DataFrame(id1 = id1, id2 = id2, b = unique_b[:,2], b_ref = unique_b[:,1],
    g = unique_g[:,2], g_ref = unique_g[:,1])